# Audio Recognition
### Tim Nguyen & Neeraj Shah
---


In the following code portions below, we implement a barebones version of how audio recognition works using the core concept sof digital signal processing.  This involves sampling the analog signal (sound input), converting it a digital signal, performing a discrete fourier transform, and matching the audio fingerprints with a predefined database.

### Dependencies

1) Need to have PyAudio library.  Install with python's package manager `pip` :

`pip install pyaudio`



In [2]:
# Import required libraries

from sys import byteorder
from array import array
import pylab
import time
import copy
#import pyaudio
import wave
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile


In [3]:
# Sample the songs in the library to create the TRUE database.



## Code below deals with audio input

Records audio and saves to a .WAV file for a specified amount of seconds.

In [ ]:
def record(fileName):
    
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100  # Sampling Rate
    RECORD_SECONDS = 10
    WAVE_OUTPUT_FILENAME = fileName

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording Audio ... ")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording Finished")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
def play(fileName):

    wf = wave.open(fileName, 'rb')

    p = pyaudio.PyAudio()

    def callback(in_data, frame_count, time_info, status):
        data = wf.readframes(frame_count)
        return (data, pyaudio.paContinue)

    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True,
                    stream_callback=callback)

    stream.start_stream()

    i = 0
    # while stream.is_active():
    while i < 125:
        time.sleep(0.1)
        i += 1

    stream.stop_stream()
    stream.close()
    wf.close()

    p.terminate()

In [ ]:
record("test.wav")  # Test record

In [ ]:
play("test.wav")

44100
22050
22050
22049


83
516


ValueError: x and y must have same first dimension

In [14]:
sample_rate, X = wavfile.read('test.wav')
print(sample_rate)
n = 44100
Fs = 44100
sec = X[:n]
secTrans = np.fft.fft(sec)/n
secTrans = secTrans[range(n/2)]

k = np.arange(n) 
T = n/Fs
frq = k/T # two sides frequency range
frq = frq[range(n/2)] # one side frequency range
print(len(secTrans))
print(len(frq))
print(max(frq))
plt.plot(frq[:1000],abs(secTrans[:1000]),'r') # plotting the spectrum
#plt.plot(11274,392,'bo')
plt.xlabel('Freq (Hz)')
plt.ylabel('|Y(freq)|')
plt.title('Frequency Amplitudes')

plt.show()
plt.grid()

big = 0
for i in range(len(secTrans[0:1000])):
    if (abs(secTrans[i]) > big):
        big = i
        
#print(secTrans[11274])
max1 = np.argmax(abs(secTrans[0:200]))
max2 = np.argmax(abs(secTrans[200:600])) + 200
#max1 = np.argmax(abs(secTrans[400:1000]))
print(max1)
print(max2)

h = signal.firwin(numtaps=n, cutoff=40, nyq=Fs/2)
y = signal.lfilter(h, 1.0, secTrans)
plt.plot(frq[:1000],y)

# plt.pcolormesh(times, frequencies, spectogram)
# plt.imshow(spectogram)
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()

## Build Database Here:

The database will store several snippets of the songs we wish to be able to recognize.
We will use a hashtable, where the key is several high frequencies and value is an array of time and song.
We will choose high frequencies between several intervals. Each key will have four frequencies.

Our frequency ranges will be from 0-40Hz, 40-80Hz, 80-120Hz, and 120Hz+
In the future, we may modify this to include more songs.
Each snippet from which we obtain frequencies will be one second long.
Since this is such a small frame of time, we can ignore time from here on out.
The only place time will be included is in the value of our hashtable. It can show us exactly where
program associated our recorded sound with our stored song. This is useful for debugging

Even with a four-point key, we still may not completely recognize a song--two sets of four frequencies may map to the same song. This is when it will useful to make multiple matches. We will track all associations and narrow the pool the longer we play the song. At the end, we should have correctly identified a song.

In [28]:
database = {}
maxChunk = 44100 #How large each of our sections are
songSnippet = []
sample_rate, X = wavfile.read('billiejeanwav.wav')
song = X[:,0]
Fs = 44100

secs = 1

print(len(song))

for i in range(len(song)):
    songSnippet.append(song[i])
    if i > secs*maxChunk:
        secs += 1
        #Know associated time passed
        # Take fft here (songSnippet)
        songSnipTrans = np.fft.fft(songSnippet)/maxChunk
        songSnipTrans = songSnipTrans[range(maxChunk/2)]
        
        
        k = np.arange(maxChunk) 
        T = maxChunk/Fs
        frq = k/T # two sides frequency range
        frq = frq[range(maxChunk/2)] # one side frequency range

        #print(secTrans[11274])
        # Find Maximums.


        max1 = np.argmax(abs(songSnipTrans[0:300]))
        max2 = np.argmax(abs(songSnipTrans[300:600])) + 300
        max3 = np.argmax(abs(songSnipTrans[600:1000])) + 600
        
        database[(max1, max2, max3)] = ('billiejeanwav.wav', secs)
        
        #Reset
        songSnippet = []
        
# ---------

songSnippet = []
sample_rate, X = wavfile.read('moon.wav')
song = X[:,0]
Fs = 44100

secs = 1

print(len(song))

for i in range(len(song)):
    songSnippet.append(song[i])
    if i > secs*maxChunk:
        secs += 1
        #Know associated time passed
        # Take fft here (songSnippet)
        songSnipTrans = np.fft.fft(songSnippet)/maxChunk
        songSnipTrans = songSnipTrans[range(maxChunk/2)]
        
        
        k = np.arange(maxChunk) 
        T = maxChunk/Fs
        frq = k/T # two sides frequency range
        frq = frq[range(maxChunk/2)] # one side frequency range

        #print(secTrans[11274])
        # Find Maximums.

        max1 = np.argmax(abs(songSnipTrans[0:300]))
        max2 = np.argmax(abs(songSnipTrans[300:600])) + 300
        max3 = np.argmax(abs(songSnipTrans[600:1000])) + 600


        
        database[(max1, max2, max3)] = ('moon.wav', secs)
        
        #Reset
        songSnippet = []
        
# -----
"""
songSnippet = []
sample_rate, X = wavfile.read('doremi.wav')
song = X[:,0]
Fs = 44100

secs = 1

print(len(song))

for i in range(len(song)):
    songSnippet.append(song[i])
    if i > secs*maxChunk:
        secs += 1
        #Know associated time passed
        # Take fft here (songSnippet)
        songSnipTrans = np.fft.fft(songSnippet)/maxChunk
        songSnipTrans = songSnipTrans[range(maxChunk/2)]
        
        
        k = np.arange(maxChunk) 
        T = maxChunk/Fs
        frq = k/T # two sides frequency range
        frq = frq[range(maxChunk/2)] # one side frequency range

        #print(secTrans[11274])
        # Find Maximums.


        max1 = np.argmax(abs(songSnipTrans[0:300]))
        max2 = np.argmax(abs(songSnipTrans[300:600])) + 300
        max3 = np.argmax(abs(songSnipTrans[600:1000])) + 600

        
        
        database[(max1, max2, max3)] = ('doremi.wav', secs)
        
        #Reset
        songSnippet = []
"""

12973871
6553728


"\nsongSnippet = []\nsample_rate, X = wavfile.read('doremi.wav')\nsong = X[:,0]\nFs = 44100\n\nsecs = 1\n\nprint(len(song))\n\nfor i in range(len(song)):\n    songSnippet.append(song[i])\n    if i > secs*maxChunk:\n        secs += 1\n        #Know associated time passed\n        # Take fft here (songSnippet)\n        songSnipTrans = np.fft.fft(songSnippet)/maxChunk\n        songSnipTrans = songSnipTrans[range(maxChunk/2)]\n        \n        \n        k = np.arange(maxChunk) \n        T = maxChunk/Fs\n        frq = k/T # two sides frequency range\n        frq = frq[range(maxChunk/2)] # one side frequency range\n\n        #print(secTrans[11274])\n        # Find Maximums.\n\n\n        max1 = np.argmax(abs(songSnipTrans[0:300]))\n        max2 = np.argmax(abs(songSnipTrans[300:600])) + 300\n        max3 = np.argmax(abs(songSnipTrans[600:1000])) + 600\n\n        \n        \n        database[(max1, max2, max3)] = ('doremi.wav', secs)\n        \n        #Reset\n        songSnippet = []\n"

6553728
12973871


In [29]:
database

{(45, 527, 890): ('moon.wav', 34),
 (46, 325, 894): ('moon.wav', 3),
 (46, 350, 697): ('moon.wav', 36),
 (46, 426, 707): ('moon.wav', 37),
 (48, 432, 663): ('moon.wav', 4),
 (60, 425, 751): ('billiejeanwav.wav', 223),
 (60, 564, 853): ('billiejeanwav.wav', 295),
 (61, 372, 753): ('billiejeanwav.wav', 47),
 (61, 421, 852): ('billiejeanwav.wav', 172),
 (61, 424, 634): ('billiejeanwav.wav', 151),
 (61, 504, 757): ('billiejeanwav.wav', 195),
 (61, 511, 792): ('billiejeanwav.wav', 34),
 (61, 562, 753): ('billiejeanwav.wav', 97),
 (61, 565, 757): ('billiejeanwav.wav', 196),
 (62, 374, 627): ('billiejeanwav.wav', 121),
 (62, 377, 754): ('billiejeanwav.wav', 105),
 (62, 379, 766): ('billiejeanwav.wav', 112),
 (62, 420, 772): ('billiejeanwav.wav', 291),
 (62, 553, 754): ('billiejeanwav.wav', 219),
 (63, 301, 753): ('billiejeanwav.wav', 108),
 (63, 376, 757): ('billiejeanwav.wav', 247),
 (63, 376, 759): ('billiejeanwav.wav', 198),
 (63, 376, 900): ('billiejeanwav.wav', 53),
 (63, 378, 755): ('bi

## The Actual Program
Keep track of "matches" and narrow song options as the song plays for a longer time. In the best case scenario, it should only need to play for the size of one song snippet.  

In [30]:
sample_rate, X = wavfile.read('test.wav')
leftChannel = X[:]
inputSnippet = []

firstTime = 1
secs = 1
associatedSongs = set()

print(len(leftChannel))

maxChunk = 44100 #How large each of our sections are

print("Beginning Music Recognition Algorithm ...\n")
for i in range(len(leftChannel)):
    
    inputSnippet.append(leftChannel[i])
    if i > secs*maxChunk:
        secs += 1
        #Know associated time passed
        # Take fft here (songSnippet)
        inputSnipTrans = np.fft.fft(inputSnippet)/maxChunk
        inputSnipTrans = inputSnipTrans[range(maxChunk/2)]
        
        
        k = np.arange(maxChunk) 
        T = maxChunk/Fs
        frq = k/T # two sides frequency range
        frq = frq[range(maxChunk/2)] # one side frequency range

        #print(secTrans[11274])
        # Find Maximums.


        max1 = np.argmax(abs(songSnipTrans[0:300]))
        max2 = np.argmax(abs(songSnipTrans[300:600])) + 300
        max3 = np.argmax(abs(songSnipTrans[600:1000])) + 600



        threshold = 20# Threshold is plus or minus 20 Hz

        associatedSongs = set(associatedSongs)
    
        if firstTime:
            #pass1 = 0 
            for i in range(2*threshold):
                first = max1 - threshold + i
                for j in range(2*threshold):
                    second = max2 - threshold + j
                    for k in range(2*threshold):
                        third = max3 - threshold + k
                        if ((first,second,third)) in database:
                            associatedSongs.add(database[(first, second, third)][0])

            print("DEBUG First : " + str(associatedSongs))

            firstTime = 0
        else:

            secondAssociatedSongs = set()
            for i in range(2*threshold):
                first = max1 - threshold + i
                for j in range(2*threshold):
                    second = max2 - threshold + j
                    for k in range(2*threshold):
                        third = max3 - threshold + k
                        if ((first, second)) in database:                
                            secondAssociatedSongs.add(database[(first, second, third)][0])
            print("DEBUG Second : " + str(secondAssociatedSongs))

            if len(secondAssociatedSongs) == 0:
                continue
            else:
                print("DEBUG Total : " + str(secondAssociatedSongs))
                associatedSongs = associatedSongs & secondAssociatedSongs

        print "DEBUG : " + str(associatedSongs)



        #store all associated songs within this array
        #add noise here--in other words, allow matches to be within plus-or-minus five percent of obtained frequency.
        #This will increase the number of matches, but will make sure we don't reject the right song (Type 2 error)
        #We don't add new matches after the first snippet of playback time. 

        #Continue to find matches till there is only one element left in the array.

        if len(associatedSongs) == 1:

            print("\nThe matched song is : %s " % (list(associatedSongs)[0]))
            break

        secs += 1
        inputSnippet = []

440320
Beginning Music Recognition Algorithm ...

DEBUG First : set(['moon.wav'])
DEBUG : set(['moon.wav'])

The matched song is : moon.wav 
